# Environmental Impact of Agricultural Practices in the World

**ADA Project Milestone 2**

This notebook consists of our initial Data Analysis of the FAOSTAT dataset on Food an agriculture. We will first study the contents of the data and its strucuture, before restructuring it in order to start our analysis. Also, some research questions initially asked will be answered by the end of this notebook.

## A. Initial Analysis

The dataset initially contained 78 csv files, but some of them were discarded as they will not be useful for our analysis. We have selected 43 CSVs that would help us with our analysis.

In [2]:
from glob import glob
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [3]:
csv_files = glob('data/**/**.csv')
len(csv_files)

43

We split those 43 csv into different directories, one for each group of csv. Each group corresponds to one category:
```.
+-- data/
|   +-- emissions_agriculture/
|      +-- ...
|   +-- emissions_land/
|      +-- ...
|   +-- environment/
|      +-- ...
|   +-- forestry/
|      +-- ...
|   +-- inputs/
|      +-- ...
|   +-- population/
|      +-- ...
|   +-- production/
|      +-- ...
```

---
### A.1 Schema consistency
---

We will first study the schemas of all the csv files we have in order to see if they are consistent or require changes

#### 1. Checking column names across whole dataset
---

Now, let's scan all the csv files and check their schemas.

In [1]:
from data_processing import scan_columns
all_columns = scan_columns(csv_files)
print("The found columns, grouped, are:\n")
for cols, f in all_columns:
    print(list(sorted(cols)), f"Num files {len(f)}")

NameError: name 'csv_files' is not defined

As we can see, sometimes the columns `area` and `areacode` are named `country` and `countrycode`, only because some csv files only contain country data, without country groups. We will rename those as to have a unified schema. Also, some files have the `note`, `elementgroup` and `months` columns. We will look into those in subsequent steps as we are now simply checking whether column naming is consistent.

In order to obtain a more consistent column naming, we will rename `country` to `area` and `countrycode` to `areacode`.

In [4]:
column_rename = {'country': 'area', 'countrycode': 'areacode'}

In [5]:
all_columns_2 = scan_columns(csv_files, column_rename)
print(f"After renaming, we obtain the following columns:\n")
for cols, f in all_columns_2:
    print(list(sorted(cols)), f"Num files {len(f)}")

After renaming, we obtain the following columns:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 34
['area', 'areacode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 4
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year', 'yearcode'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'note', 'unit', 'value', 'year', 'yearcode'] Num files 4


#### 2. Checking which columns to drop
---

Now we have a few files that have different schemas. One column that we should look into before continuing is `note`, as it is in 4 files.

In [6]:
from data_processing import get_column_unique_values

files_with_note = all_columns_2[-1][1]
note_values = get_column_unique_values(files_with_note, column_rename, None,['note'])
note_values

,note
0,NaN


As we can see, all values for this column are NaN, so we can safely drop the column.

#### 3. Checking duplicate columns
---

We figured it would be useful to scan for duplicate columns in each dataframe (i.e. columns with different names but same values)

In [7]:
from data_processing import scan_column_duplicates
duplicates = scan_column_duplicates(csv_files, column_rename)
for c, f in duplicates:
    print(f"Duplicates for {c} in {len(f)} files")

Duplicates for [('yearcode', 'year')] in 39 files
Duplicates for [('elementgroup', 'elementcode')] in 3 files


As we can see, most files have `year` and `yearcode` columns which are equal. Hence, we can safely drop this column. However, for `elementgroup` and `elementcode`, they are equal in almost all CSV where they appear (3/4), but not all, so we cannot safely drop it without checking. We choose to keep `elementcode` when those two are equal, and keep them both when they are not.

Hence, we can define a list of columns to be checked and dropped; We drop them if they fulfill any of the following conditions:
 - NaN in all rows
 - Duplicate with another column

In [8]:
check_columns = ["note", "yearcode", "elementgroup"]
all_columns_3 = scan_columns(csv_files, column_rename, check_columns)
print(f"After renaming and dropping columns, we obtain the following columns:\n")
for cols, f in all_columns_3:
    print(list(sorted(cols)), f"Num files {len(f)}")

After renaming and dropping columns, we obtain the following columns:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 41
['area', 'areacode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year'] Num files 1


We now have 41 files with identical schemas, and 2 files that have a different one:
 - The file containing the `elementgroup` additional column, which refers to groups of `elements`
 - The file with monthly data and no `item` and `itemcode` columns
 
To obtain the desired format, we can now call `load_dataframe(<file>, column_rename, drop_columns)` with `column_rename = {'country': 'area', 'countrycode': 'areacode'}` and `drop_columns = ["note", "yearcode", "elementgroup"]`

---
### A.2 Schema description

---
Now that we have a unified schema for (almost) all csv files, we can start looking into the meaning of each column and their possible value. 

#### 1. Area columns
---
We will first look into the columns `area` and `areacode`. According to FAOSTAT's website, each area is defined by a unique areacode, however some areas include other ones, i.e. there are grouped areas in the datasets. We would expect a one-to-one mapping between those two columns. Let's see how this looks like

To verify that it is indeed a one-to-one mapping, we will append all values from all csv files, and drop duplicates. Then we group by area and see if the length of the group is 1.

In [10]:
from utils import is_unique_mapping

area_values = get_column_unique_values(csv_files, column_rename, check_columns, ['area', 'areacode'])
is_unique_mapping(area_values, 'area', 'areacode')

False

It seems that the mapping is not one-to-one. Let's look into why that is. First we group by `areacode` and aggregate to a list of `area` and then we do the inverse. This way, we can see how the mapping is.

In [11]:
grpd = area_values.groupby('area')['areacode'].apply(list)
grpd[grpd.apply(lambda x: len(x) > 1)]

Series([], Name: areacode, dtype: object)

The mapping `area` -> `areacode` is unique, now for the opposite:

In [12]:
grpd = area_values.groupby('areacode')['area'].apply(list)
grpd[grpd.apply(lambda x: len(x) > 1)]

areacode
35                               [Cabo Verde, Cape Verde]
151     [Netherlands Antilles (former), Netherlands An...
154     [The former Yugoslav Republic of Macedonia, No...
167                             [Czechia, Czech Republic]
209                                 [Eswatini, Swaziland]
299           [Occupied Palestinian Territory, Palestine]
5000                             [World, World + (Total)]
5100                           [Africa, Africa + (Total)]
5101           [Eastern Africa, Eastern Africa + (Total)]
5102             [Middle Africa, Middle Africa + (Total)]
5103         [Northern Africa, Northern Africa + (Total)]
5104         [Southern Africa, Southern Africa + (Total)]
5105           [Western Africa, Western Africa + (Total)]
5200                       [Americas, Americas + (Total)]
5203       [Northern America, Northern America + (Total)]
5204         [Central America, Central America + (Total)]
5206                     [Caribbean, Caribbean + (Total)]
5207 

It seems that the same `areacode` is sometimes mapped to multiple `area`, most of the time by adding "+ (Total)". For that, we could drop the `area` column and keep working with `areacode`.

However, we still need a fixed database that maps the `areacode` to `area`, and for this, FAOSTAT provides one CSV file containing the mapping, as well as another one containing a description of country groups (i.e. "Asia") and which contries they contain.

In [14]:
from data_processing import load_dataframe
area_codes = load_dataframe('data/countries.csv')
area_codes.head()

,countrycode,country,m49code,iso2code,iso3code,startyear,endyear
0,2,Afghanistan,4.0,AF,AFG,NaN,NaN
1,5100,Africa,2.0,NaN,X06,NaN,NaN
2,284,Ã land Islands,248.0,NaN,ALA,NaN,NaN
3,3,Albania,8.0,AL,ALB,NaN,NaN
4,4,Algeria,12.0,DZ,DZA,NaN,NaN


In [15]:
country_groups = load_dataframe('data/country_groups.csv')
country_groups.head()

,countrygroupcode,countrygroup,countrycode,country,m49code,iso2code,iso3code
0,5100,Africa,4,Algeria,12.0,DZ,DZA
1,5100,Africa,7,Angola,24.0,AO,AGO
2,5100,Africa,53,Benin,204.0,BJ,BEN
3,5100,Africa,20,Botswana,72.0,BW,BWA
4,5100,Africa,24,British Indian Ocean Territory,86.0,IO,IOT


Here, countries are grouped into multiple `countrygroup`, so we know exactly of which countries each group is formed. These country groups are present in the dataset as `area`, meaning there are aggregated values in the dataset. For example: we can find the emissions for "Algeria" and for "Africa", where the latter is an aggregated value over the whole group. We will need to be careful when aggregating values in the future, as we could account multiple times for one country.

In [16]:
is_unique_mapping(area_codes[['country', 'countrycode']].drop_duplicates(), 'country', 'countrycode')

True

In [17]:
all(x in area_codes.countrycode.unique() for x in area_values.areacode.unique())

True

Hence, we can use the dataframe `area_codes` to obtain the country's name from it's code, and `country_groups` to get which countries form a group. For this, we can now drop the `area` column in our dataset.

#### 2. Element Columns
---

The `element` and `elementcode` represent the measure quantity for a given `item`. A quantity has a name and a unit, which is why we believe these two columns should also have a one-to-one mapping accross the whole dataset. Also, since an `elementcode` potentially uniquely identifies (`element`, `unit`) pair, we might drop those two columns as to make the csv files smaller and easier to manipulate.

First let's check if indeed this mapping is one-to-one:

In [18]:
element_values = get_column_unique_values(csv_files, column_rename, check_columns, ['elementcode', 'element', 'unit'])
is_unique_mapping(element_values, 'elementcode', ['element', 'unit'])

True

In [19]:
element_values.head()

,elementcode,element,unit
0,5111,Stocks,Head
171,5112,Stocks,1000 Head
684,5114,Stocks,No
0,5510,Production,tonnes
0,5313,Laying,1000 Head


As we can see, `elementcode` uniquely identify (`element`, `unit`) pairs, so we can safely drop those two columns and only use `elementcode`. We will later pivot each csv as to obtain all the `elementcode`s as columns, so we can reduce de number of rows significantly. A mapping using a dictionnary will of course be necessary in order to have a nice GUI where users can select the (element, unit) pair instead of the code.

#### 3. Item columns
---

According to FAOSTAT, the `item` and `itemcode` columns represent item on which measurements were done. For example an item can be `cattle` and the measurement can be "CH4 emissions in gigagrams". 
Similarly to what we did above, we expect `item` and `itemcode` to have a one-to-one relationship. Let's verify this using the same functions

In [20]:
item_values = get_column_unique_values(csv_files, column_rename, check_columns, ['item', 'itemcode'], with_file=True)
is_unique_mapping(item_values[['item', 'itemcode']], 'item', 'itemcode')

False

It seems that `item` to `itemcode` is not unique for a few items, let's check those and try to understand why it is the case.

In [21]:
grpd = item_values.groupby('itemcode')['item'].agg(set)
grpd[grpd.apply(len) > 1]

Series([], Name: item, dtype: object)

The mapping `itemcode` -> `item` is unique, now let's check the other way around

In [22]:
grouped = item_values.groupby('item')['itemcode'].agg(set)
grouped[grouped.apply(len) > 1]

item
Ammonium nitrate (AN)                                  {1362, 4003}
Ammonium sulphate                                      {1361, 4002}
Burning - all categories                               {6795, 6798}
Cattle                                                  {866, 1757}
Chickens                                               {1057, 1054}
Cropland                                               {6620, 5070}
Disinfectants                                          {1358, 1351}
Forest land                                      {5065, 6749, 6646}
Grassland                                              {6794, 6983}
Mineral Oils                                           {1354, 1316}
Other Pesticides nes                                   {1355, 1359}
Other nitrogenous fertilizers, n.e.c.                  {4008, 1369}
Other potassic fertilizers, n.e.c.                     {4018, 1391}
Plant Growth Regulators                                {1356, 1341}
Potassium sulphate (sulphate of potash) (SO

Some items seem to have multiple (up to 3) different item codes, which doesn't seem very normal. 

**a.** It seems that some of those items correspond to nutrients provided throught fertilizers. Let's see in which files those appear. The items related to nutrients are the following :

In [23]:
nutrient_items = ["Ammonium nitrate (AN)", "Ammonium sulphate", "Other nitrogenous fertilizers, n.e.c.", "Other potassic fertilizers, n.e.c.", "Potassium sulphate (sulphate of potash) (SOP)", "Urea"]
item_values[item_values.item.isin(nutrient_items)].file.unique()

array(['data/inputs/Inputs_FertilizersArchive_E_All_Data_(Normalized).csv',
       'data/inputs/Inputs_FertilizersProduct_E_All_Data_(Normalized).csv'],
      dtype=object)

Those items are only present in two dataframes, related to Ferilizer use. If we look closely at the names of csv files, we can see that one of them is "Archive", 
while the other is product. After having looking and reading the documentation for this CSV http://www.fao.org/faostat/en/#data/RA, we understand that it is an archive document, that has not been updated since 2002. 

Before that date, all data on Fertilizer was put into one single dataframe (i.e. info about production, trade and consumption) both in total nutrients and amount of product. After 2002, it was split into nutrient and product, which is why we have two `itemcode` values for those `items`. 

Hence, we should be careful when studying the fertilizer use, as data collection has changed in 2002.


**b.** Some items are related to pesticides. Let's in which csv files they appear

In [24]:
pesticide_items = ["Disinfectants", "Mineral Oils", "Other Pesticides nes", "Plant Growth Regulators"]
item_values[item_values.item.isin(pesticide_items)].file.unique()

array(['data/inputs/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv'],
      dtype=object)

Some duplicate items with the same name but different codes appear in only one csv file, which seems quite odd. We suspect having duplicate rows in that case. Let's check that out.

In [26]:
from data_processing import check_duplicate_items
t = load_dataframe('data/inputs/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv', column_rename, check_columns)
check_duplicate_items(t, pesticide_items)

Duplicate item for Disinfectants codes [1358 1351]
Duplicate item for Mineral Oils codes [1354 1316]
Duplicate item for Other Pesticides nes codes [1359 1355]
Duplicate item for Plant Growth Regulators codes [1356 1341]


We now know that there are potentially duplicate rows for the same `item` but a different `itemcode` (with the same measurements). Hence, we need to add a functionnality that checks for duplicated items in the dataframe.

**c.** Now let's look at livestock items

In [27]:
livestock_items = ["Cattle", "Chickens"]
item_values[item_values.item.isin(livestock_items)].groupby(['item', 'itemcode']).agg(set)

file
item     itemcode                                                   
Cattle   866       {data/production/Production_Livestock_E_All_Da...
         1757      {data/environment/Environment_LivestockManure_...
Chickens 1054      {data/emissions_agriculture/Emissions_Agricult...
         1057      {data/production/Production_Livestock_E_All_Da...

The `itemcode` -> `item` seems one-to-one in each csv file, but we see that there are two distinct `itemcode` for "Cattle" and "Chickens" accross all csv files.

In [28]:
item_values[item_values.item.isin(livestock_items)][['file', 'itemcode', 'item']].drop_duplicates().values

array([['data/production/Production_Livestock_E_All_Data_(Normalized).csv',
        866, 'Cattle'],
       ['data/production/Production_Livestock_E_All_Data_(Normalized).csv',
        1057, 'Chickens'],
       ['data/environment/Environment_LivestockPatterns_E_All_Data_(Normalized).csv',
        866, 'Cattle'],
       ['data/environment/Environment_LivestockPatterns_E_All_Data_(Normalized).csv',
        1057, 'Chickens'],
       ['data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv',
        1757, 'Cattle'],
       ['data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv',
        1054, 'Chickens'],
       ['data/emissions_agriculture/Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Normalized).csv',
        1757, 'Cattle'],
       ['data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).csv',
        1757, 'Cattle'],
       ['data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(N

It seems that different values for `itemcode` for both "Chickens" and "Cattles" differ when the measured quantity is realted to Manure. Thus, this might be a discrepancy in the data as it does not appear for other items. However, since the two different item codes appear in un-related csv files, we can keep them that way, and have two itemcodes that map to "Chickens" and "Cattle"

**d.** And finally, land related items

In [29]:
land_items = ["Forest land"]
item_values[item_values.item.isin(land_items)].sort_values(['item', 'file']).file.values

array(['data/emissions_land/Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).csv',
       'data/emissions_land/Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).csv',
       'data/emissions_land/Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv',
       'data/environment/Environment_LandUse_E_All_Data_(Normalized).csv',
       'data/inputs/Inputs_LandUse_E_All_Data_(Normalized).csv'],
      dtype=object)

After looking into the description of each csv file, we have found the following explanations:
- For `Burning - all categories`:
    - for "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).csv", it represents the emissions due to burning of Savanna (all categories) i.e. aggregated over all items
    - for "Emissions_Land_Use_Burning_Biomass_E_All_Data_(Normalized).csv", it represents the emissions due to the burning of Biomass.
- For `Grassland`: 
    - for "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).csv", it represents the emissions burning of grassland
    - for "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv" it represents the Green house gass emissions and removals from grasslands
    - for "Environment_LandCover_E_All_Data_(Normalized).csv" it represents the area of grassland
- For `Cropland`:
    - for "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv" it represents the Green house gass emissions and removals from Croplands
    - for "Environment_LandUse_E_All_Data_(Normalized).csv" it represents the area of Cropland
    - for "Inputs_LandUse_E_All_Data_(Normalized).csv" it represents the usage of Cropland
    
TODO : Add Forest Land, 

#### 4. Flag Columns
---

According to FAOSTAT, not all the data points are official figures reported by individual countries. Some are estimates, others are aggregates done by external entities. In order to understand the meaning of those flagas, they provide a csv file containing the description of each.

In [30]:
flags = load_dataframe("data/flags.csv")
flags.head()

,flag,flags
0,NaN,Official data
1,*,Unofficial figure
2,A,"Aggregate, may include official, semi-official..."
3,B,Balance
4,Bk,Break in series


In [31]:
print(f"There are {len(flags)} flags")

There are 41 flags


---
### A.3 Conclusion about Schemas

---

After having applied some analysis on the initially different schemas of the dataframes, we are now able to obtain identitcal schemas for 41 of them, and left two of them where we believe they should not be changed as some information could be lost.

The common schema has the following columns:

`['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year']`

For now, we have come up with a few steps that should be applied when loading a dataframe:
 - Rename columns to remove spaces and captial letters
 - Rename "Country" to "Area" and "CountryCode" to "Areacode"
 - Drop columns that are either duplicates or NaN values
 - Rename "gigagrams" to "Gigagrams", as otherwise we would have a non one-to-one mapping for `elementcode`
 - Check for items if they have duplicate rows.

The function that does the first 4 cleaning steps is `data_processing/load_dataframe`, and the one that puts everything together is `data_processing/load_clean_dataframe`.

Also, we have decided to only work with codes (i.e. `areacode` and `elementcode`) and drop `area`, `element` and `unit`. Of course, we will need a mapping from `code` -> `value`. Additionally, in order to keep track of `countrygroup` and do know how values are aggregated, we will also create a mapping that tells us which countries form a group, by codes.

Hence, to load a dataframe, we simply call :
```
df = load_clean_dataframe(<filename>, column_rename, check_columns, drop_columns)
```

with  `column_rename = {'country': 'area', 'countrycode': 'areacode'}` and `check_columns = ["note", "yearcode", "elementgroup"]` and `drop_columns = ["area", "item", "element", "unit"]`

In [4]:
column_rename = {'country': 'area', 'countrycode': 'areacode'}
check_columns = ["note", "yearcode", "elementgroup"]
drop_columns = ["area", "item", "element", "unit"]

Now we need to get the mappings for :
- `elementcode` -> (`element`, `unit`) (unique in both ways)
- `itemcode` -> `item` (unique in that way, but not the other way around)
- `areacode` -> `area` (unique in both ways)

In [5]:
from mappings import get_mapping, get_area_mapping, get_country_groups
item_mapping = get_mapping(csv_files, column_rename, check_columns, ['itemcode','item'])
element_mapping = get_mapping(csv_files, column_rename, check_columns, ['elementcode', 'element', 'unit'])
area_mapping = get_area_mapping("data/countries.csv")
country_groups = get_country_groups("data/country_groups.csv")

---
---
## B. Reshaping the Database

For now everything was stored in database format (i.e. Row format), but we would like to have elements in columns, so that we can reduce drastically the number of rows, and so we can have all the measurements for a single (`itemcode`, `areacode`, `year`) pair.

---
### B.1 Merging all files with same schema

---

Before reshaping, we know that 41 csv files have the same schema. Thus, it would be good to have them all in a single dataframe in order to facilitate access to values. We will then create a single dataframe from all those 41 (over 43 in total). This dataframe will then have potentially duplicate rows, which we will drop.

In [6]:
from data_processing import load_clean_dataframe
def load_all_df_with_schema(files, schema):
    dfs = []
    for f in csv_files:
        df = load_clean_dataframe(f, column_rename, check_columns, drop_columns) # Load the DF
        if all(x in schema for x in df.columns):  # Check if schema corresponds
            df = df.assign(file=f)
            dfs.append(df)
    df = pd.concat(dfs).reset_index(drop=True)
    shape = df.shape[0]
    
    to_keep = df.drop(columns=['file']).drop_duplicates().index # Drop all duplicated row, after removing `file`
    df = df.loc[to_keep].reset_index(drop=True)
    print(f"Dropped {shape - df.shape[0]} duplicate rows")
    return df

In [7]:
df = load_all_df_with_schema(csv_files, ['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'])

Dropped duplicate item Disinfectants with codes [1358, 1351] (Dropped 1351)
Dropped duplicate item Mineral Oils with codes [1354, 1316] (Dropped 1316)
Dropped duplicate item Other Pesticides nes with codes [1359, 1355] (Dropped 1355)
Dropped duplicate item Plant Growth Regulators with codes [1356, 1341] (Dropped 1341)
Dropped 1919040 duplicate rows


In [8]:
df.head()

,areacode,itemcode,elementcode,year,value,flag,file
0,2,1107,5111,1961,1300000.0,NaN,data/production/Production_Livestock_E_All_Dat...
1,2,1107,5111,1962,851850.0,NaN,data/production/Production_Livestock_E_All_Dat...
2,2,1107,5111,1963,1001112.0,NaN,data/production/Production_Livestock_E_All_Dat...
3,2,1107,5111,1964,1150000.0,F,data/production/Production_Livestock_E_All_Dat...
4,2,1107,5111,1965,1300000.0,NaN,data/production/Production_Livestock_E_All_Dat...


In [9]:
group_codes = set(country_groups.keys())
df = df[~df.areacode.isin(group_codes)]#.reset_index(drop=True)  # Drop all country groups

In [87]:
import gc
gc.collect()

65

---
### B.2 Checking for duplicate measurements

---

Ideally, we would like to have a single row for each (`areacode`, `itemcode`, `elementcode` and `year`), i.e one measurement. Let's check if it is the case:

In [11]:
grouped = df.groupby(['areacode', 'itemcode', 'elementcode', 'year'])[['value', 'flag']].nunique(dropna=False).reset_index()


In [12]:
grouped['value'].value_counts(dropna=False) / grouped.shape[0]

1    0.964959
2    0.035041
Name: value, dtype: float64

As we can see, it is not the case:
    - Most of the time there is one measurement (~96%)
    - Some have two measurements (~4%)
This could be due to different `value` and/or different `flag`

Let's look at those values

#### 1. Duplicated Measurements

To study the differences, we will look at both flags and values. A `NaN` flag means "Official Figure"

In [13]:
indexed = df.set_index(['areacode', 'itemcode', 'elementcode', 'year']) # Set the index to what what should be unique
duplicated_1 = indexed.index.duplicated(keep='first')  # Take the first duplicate
duplicated_2 = indexed.index.duplicated(keep='last')  # Take the second one
joined = indexed.loc[duplicated_1].join(indexed.loc[duplicated_2], rsuffix='_2')  # Join on the index to have both values and indices

In [92]:
joined.groupby(['file', 'file_2']).apply(lambda x: pd.Series({'mean_diff': np.mean(x['value'] - x['value_2']), 'std': np.std(x['value'] - x['value_2']), 'num': x.shape[0]})).sort_values('mean_diff')

,,mean_diff,std,num
file,file_2,,,
data/environment/Environment_Emissions_intensities_E_All_Data_(Normalized).csv,data/production/Production_Crops_E_All_Data_(Normalized).csv,-1.266076e+05,710051.507911,212.0
data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv,data/production/Production_Livestock_E_All_Data_(Normalized).csv,-5.714286e-02,0.998366,70.0
data/emissions_agriculture/Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).csv,data/emissions_land/Emissions_Land_Use_Grassland_E_All_Data_(Normalized).csv,0.000000e+00,0.000000,21.0
data/inputs/Inputs_LandUse_E_All_Data_(Normalized).csv,data/emissions_land/Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).csv,0.000000e+00,0.000000,6290.0
data/emissions_agriculture/Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Normalized).csv,data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv,1.437313e-07,0.000032,221474.0
data/emissions_agriculture/Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Normalized).csv,data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv,4.364877e-07,0.000031,345685.0
data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).csv,data/environment/Environment_LivestockManure_E_All_Data_(Normalized).csv,8.953654e-07,0.000033,49992.0
data/emissions_agriculture/Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).csv,data/emissions_land/Emissions_Land_Use_Cropland_E_All_Data_(Normalized).csv,5.588237e-05,0.000259,136.0
data/emissions_agriculture/Emissions_Agriculture_Agriculture_total_E_All_Data_(Normalized).csv,data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv,8.761786e+01,2234.692735,11964.0


In [98]:
def get_percentage_diff(value_1, value_2):
    diff = np.abs(value_1 - value_2)
    sum_ = (value_1 + value_2) / 2.0
    diff = diff / sum_
    return diff

def is_close(value_1, value_2, threshold=0.01):
    diff = get_percentage_diff(value_1, value_2)
    return (value_1 == value_2) | (diff < threshold) | (value_1.isna() & value_2.isna())

def is_same_flag(flag_1, flag_2):
    return (flag_1 == flag_2) | (flag_1.isna() & flag_2.isna())

In order to remove thos duplicates, we will first compute the difference between the two values. 
- If it is lower than a certain percentage (~1%) we return the mean of both values.
- If it is higher than 1%, then we need to look at the flags and determine a hierarchy that will choose a value based on its flag

In [93]:
less_than_thresh = joined[(is_close(joined.value, joined.value_2)) & is_same_flag(joined.flag, joined.flag_2)].shape[0] / joined.shape[0]
print("{:.2f}% of duplicated rows have a difference less than 1% and the same flag".format(less_than_thresh*100))

97.22% of duplicated rows have a difference less than 1% and the same flag


In [100]:
close_same_flag = joined[is_close(joined.value, joined.value_2) & is_same_flag(joined.flag, joined.flag_2)]
close_same_flag.groupby(['file', 'file_2']).apply(lambda x: pd.Series({'mean_diff': np.mean(get_percentage_diff(x['value'], x['value_2'])*100), 'std': np.std(x['value'] - x['value_2']), 'num': x.shape[0]})).sort_values('mean_diff')

mean_diff  \
file                                               file_2                                                             
data/emissions_agriculture/Emissions_Agricultur... data/emissions_land/Emissions_Land_Use_Cropland...  1.858688e-07   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...  6.152365e-06   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...  9.461075e-06   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...  1.978741e-05   
data/environment/Environment_LivestockManure_E_... data/production/Production_Livestock_E_All_Data...  5.809386e-03   
data/production/Production_Crops_E_All_Data_(No... data/production/Production_CropsProcessed_E_All...  6.328335e-02   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_Emissions_by_Secto...  2.321725e-01   
data/environment/Environment_Emissions_intensit... data/production/Production_Crops_E_All_Data_(No...  2.750057e-01   
                                                   data/production/Production_LivestockPrimary_E_A...  3.226978e-01   
data/emissions_land/Emissions_Land_Use_Land_Use... data/environment/Environment_Emissions_by_Secto...  5.266895e-01   

                                                                                                                 std  \
file                                               file_2                                                              
data/emissions_agriculture/Emissions_Agricultur... data/emissions_land/Emissions_Land_Use_Cropland...       0.000280   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...       0.000032   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...       0.000033   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...       0.000031   
data/environment/Environment_LivestockManure_E_... data/production/Production_Livestock_E_All_Data...       0.998366   
data/production/Production_Crops_E_All_Data_(No... data/production/Production_CropsProcessed_E_All...    2956.769172   
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_Emissions_by_Secto...     278.670721   
data/environment/Environment_Emissions_intensit... data/production/Production_Crops_E_All_Data_(No...  182584.427695   
                                                   data/production/Production_LivestockPrimary_E_A...   12503.126488   
data/emissions_land/Emissions_Land_Use_Land_Use... data/environment/Environment_Emissions_by_Secto...      26.556543   

                                                                                                            num  
file                                               file_2                                                        
data/emissions_agriculture/Emissions_Agricultur... data/emissions_land/Emissions_Land_Use_Cropland...     115.0  
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...  221474.0  
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...   49992.0  
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_LivestockManure_E_...  345685.0  
data/environment/Environment_LivestockManure_E_... data/production/Production_Livestock_E_All_Data...      70.0  
data/production/Production_Crops_E_All_Data_(No... data/production/Production_CropsProcessed_E_All...      95.0  
data/emissions_agriculture/Emissions_Agricultur... data/environment/Environment_Emissions_by_Secto...    6846.0  
data/environment/Environment_Emissions_intensit... data/production/Production_Crops_E_All_Data_(No...      87.0  
                                                   data/production/Production_LivestockPrimary_E

In [103]:
close_same_flag[close_same_flag.file == "data/emissions_land/Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv"]

value flag  \
areacode itemcode elementcode year                 
144      1707     7243        2007    545.0    A   
                  7244        2007   1250.0    A   
168      1707     7243        2002   1016.0    A   
251      1707     7243        2005  13583.0    A   

                                                                                 file  \
areacode itemcode elementcode year                                                      
144      1707     7243        2007  data/emissions_land/Emissions_Land_Use_Land_Us...   
                  7244        2007  data/emissions_land/Emissions_Land_Use_Land_Us...   
168      1707     7243        2002  data/emissions_land/Emissions_Land_Use_Land_Us...   
251      1707     7243        2005  data/emissions_land/Emissions_Land_Use_Land_Us...   

                                    value_2 flag_2  \
areacode itemcode elementcode year                   
144      1707     7243        2007    548.0      A   
                  7244        2007   1262.0      A   
168      1707     7243        2002   1018.0      A   
251      1707     7243        2005  13528.0      A   

                                                                               file_2  
areacode itemcode elementcode year                                                     
144      1707     7243        2007  data/environment/Environment_Emissions_by_Sect...  
                  7244        2007  data/environment/Environment_Emissions_by_Sect...  
168      1707     7243        2002  data/environment/Environment_Emissions_by_Sect...  
251      1707     7243        2005  data/environment/Environment_Emissions_by_Sect...

In [111]:
item_mapping[1707]

'Land Use total'

In [101]:
close_diff_flag = joined[is_close(joined.value, joined.value_2) & ~ is_same_flag(joined.flag, joined.flag_2)]
close_diff_flag.groupby(['file', 'file_2']).apply(lambda x: pd.Series({'mean_diff': np.mean(get_percentage_diff(x['value'], x['value_2'])*100), 'num': x.shape[0]})).sort_values('mean_diff')

,,mean_diff,num
file,file_2,,
data/emissions_agriculture/Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).csv,data/emissions_land/Emissions_Land_Use_Grassland_E_All_Data_(Normalized).csv,0.000000e+00,21.0
data/inputs/Inputs_LandUse_E_All_Data_(Normalized).csv,data/emissions_land/Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).csv,0.000000e+00,6290.0
data/emissions_agriculture/Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).csv,data/emissions_land/Emissions_Land_Use_Cropland_E_All_Data_(Normalized).csv,4.931485e-10,21.0
data/environment/Environment_Emissions_intensities_E_All_Data_(Normalized).csv,data/production/Production_LivestockPrimary_E_All_Data_(Normalized).csv,1.604230e-01,119.0
data/production/Production_Crops_E_All_Data_(Normalized).csv,data/production/Production_CropsProcessed_E_All_Data_(Normalized).csv,3.619859e-01,20.0
data/environment/Environment_Emissions_intensities_E_All_Data_(Normalized).csv,data/production/Production_Crops_E_All_Data_(Normalized).csv,3.905679e-01,2.0


In [102]:
far_same_flag = joined[~is_close(joined.value, joined.value_2) & is_same_flag(joined.flag, joined.flag_2)].reset_index()
far_same_flag.groupby(['file', 'file_2']).apply(lambda x: pd.Series({'mean_diff': np.mean(get_percentage_diff(x['value'], x['value_2'])*100), 'num': x.shape[0]})).sort_values('mean_diff')

,,mean_diff,num
file,file_2,,
data/environment/Environment_Emissions_intensities_E_All_Data_(Normalized).csv,data/production/Production_Crops_E_All_Data_(Normalized).csv,10.585788,93.0
data/emissions_agriculture/Emissions_Agriculture_Agriculture_total_E_All_Data_(Normalized).csv,data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv,13.550441,5118.0
data/environment/Environment_Emissions_intensities_E_All_Data_(Normalized).csv,data/production/Production_LivestockPrimary_E_All_Data_(Normalized).csv,16.292990,1525.0
data/production/Production_Crops_E_All_Data_(Normalized).csv,data/production/Production_CropsProcessed_E_All_Data_(Normalized).csv,17.697286,86.0
data/emissions_land/Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).csv,data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv,100.608908,4004.0


In [112]:
df[df.file == "data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv"]

,areacode,itemcode,elementcode,year,value,flag,file
7581180,2,6814,7231,1990,943.2945,A,data/environment/Environment_Emissions_by_Sect...
7581181,2,6814,7231,1991,1022.1110,A,data/environment/Environment_Emissions_by_Sect...
7581182,2,6814,7231,1992,542.7868,A,data/environment/Environment_Emissions_by_Sect...
7581183,2,6814,7231,1993,548.8871,A,data/environment/Environment_Emissions_by_Sect...
7581184,2,6814,7231,1994,537.9067,A,data/environment/Environment_Emissions_by_Sect...
...,...,...,...,...,...,...,...
8276414,181,6825,7180,2006,0.0000,Fc,data/environment/Environment_Emissions_by_Sect...
8276415,181,6825,7180,2007,0.0000,Fc,data/environment/Environment_Emissions_by_Sect...
8276416,181,6825,7180,2008,0.0000,Fc,data/environment/Environment_Emissions_by_Sect...
8276417,181,6825,7180,2009,0.0000,Fc,data/environment/Environment_Emissions_by_Sect...


In [ ]:
joined[(joined.file == "data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv")]

In [37]:
far_diff_flag = joined[~is_close(joined.value, joined.value_2) & ~is_same_flag(joined.flag, joined.flag_2)]
far_diff_flag.groupby(['file', 'file_2']).apply(lambda x: pd.Series({'mean_diff': np.mean(np.abs(x['value'] - x['value_2'])), 'num': x.shape[0]})).sort_values('mean_diff')

mean_diff  \
file                                               file_2                                                              
data/production/Production_Crops_E_All_Data_(No... data/production/Production_CropsProcessed_E_All...   23287.963636   
data/environment/Environment_Emissions_intensit... data/production/Production_LivestockPrimary_E_A...   94892.984615   
                                                   data/production/Production_Crops_E_All_Data_(No...  604781.933333   

                                                                                                         num  
file                                               file_2                                                     
data/production/Production_Crops_E_All_Data_(No... data/production/Production_CropsProcessed_E_All...  111.0  
data/environment/Environment_Emissions_intensit... data/production/Production_LivestockPrimary_E_A...  520.0  
                                                   data/production/Production_Crops_E_All_Data_(No...   30.0